In [1]:
import numpy as np
import scipy as sp

In [2]:
def expression_matrix(n, p):
    
    X = np.random.randn(n, p)
    #print(X)



    col_means = np.mean(X, axis=0)
    
    col_std = np.std(X, axis=0)
    
    X_normalized = (X - col_means) / col_std #We use the  normalization to have a zero mean and a unit variance 
    
    return X_normalized

In [3]:
p = 5
n = 50   

X = expression_matrix(n, p)
y = np.random.randint(2, size=p)
print(y)

print(f'Normalized values: {X}')
print(f"standard deviation: {np.std(X, axis=0)}")

[1 1 1 1 0]
Normalized values: [[-0.22415379  0.9824532  -1.04207291  1.98257619  0.06676293]
 [-1.58785224  0.05736811 -1.20699088  0.52904362 -1.32008552]
 [-0.1340121  -1.16095211 -0.53462852  1.43142326  0.15772313]
 [ 0.72105361  0.16500347 -0.75299928  1.15135094  0.88530279]
 [-0.20877131 -0.45843592  1.61777696 -1.55294256  0.37115795]
 [-0.99567112 -0.16649615 -1.09058117  2.03123517 -0.86625139]
 [ 1.13704369  0.25487417 -0.91235904 -0.59792545  2.23599801]
 [-1.13456187  1.237676    1.99217237  1.48278643 -0.09784065]
 [-0.03592978 -0.98125711  0.12459876  0.67808298  1.53652501]
 [-0.57448606 -0.32283453 -1.55508666 -0.2201472   0.01771123]
 [ 0.02078071 -0.72425963  0.31012189 -1.59367588  1.01687231]
 [-0.95269321  0.65927895  0.50063452  0.33306993  1.50585532]
 [-0.7793643  -1.27282174  0.73343685  0.56456089 -0.11661656]
 [-1.13047342  0.45176466 -0.37835812 -0.19480386  0.90084378]
 [ 1.16450001 -0.98860823  0.26526913 -0.70767072  0.01547212]
 [-0.04066668  0.0504129

In [4]:
def score(X, y):

    y0 = []
    y1 = []
    for index, j in enumerate(y):
        print(j)
        if j == 1:
            y1 = np.append(y1 ,index)
    
    y1 = np.asarray(y1, dtype = 'int')
    score_of_x = np.zeros(n)
        
    
    for i in range(p):
        if i not in y1:
            y0 = np.append(y0, i)
    y0 = np.asarray(y0, dtype = 'int')

                    
    for index, row in enumerate(X):
        for i in y1:
            for j in y0:
                if row[i] >= row[j]:
                    score_of_x[index] = score_of_x[index] + 1
       
    return score_of_x,y0,y1

In [5]:
scoreX,y0,y1 = score(X, y)

1
1
1
1
0


In [6]:
def flip_sign(X, score):
    s_max = np.max(score)
    flipped_X = []
    
    for i in range(len(score)):
        if(score[i] > s_max/2):
            X[i] = X[i] * (-1)

    return X
        


In [7]:
X = flip_sign(X, scoreX)

In [8]:
def new_score(score):
    new_score = []
    s_max = np.max(score)
    for i in range(len(score)):
        n = np.minimum(int(score[i]), int(s_max)-int(score[i]))
        new_score.append(n)
    return new_score
        

In [9]:
newscore = new_score(scoreX)

In [10]:
def margin(X,y0,y1):
    m = []
    max_x0 = []
    min_x1 = []
    
    for i in range(X.shape[0]):
        X0 = []
        X1 = []
        for j in y0:
            X0.append(X[i][j])
        for z in y1:
            X1.append(X[i][z])
            
            
        max_x0.append(np.max(X0))
        min_x1.append(np.min(X1))
        
        m.append(min_x1[i] -max_x0[i])
        
    return m


In [11]:
def init_cluster(new_score, X, y0, y1):
    min_score = np.min(new_score)
    similar_genes = []
    margins = []
    for i in range(len(new_score)):
        if (new_score[i] == min_score):
            similar_genes.append(i)
            
    marginz = margin(X, y0, y1)
    for i in similar_genes:
        margins.append(marginz[i])
        
    max_margin = np.argmax(margins)
        
    ie = similar_genes[max_margin]
    
    return {
        "ie" : ie,
        "cg" : X[ie] 
    } 
        

In [12]:
c = []
c.append(init_cluster(newscore, X, y0, y1))

In [13]:
c

[{'ie': 38,
  'cg': array([-0.41941772,  0.33204502,  0.53970854, -0.03042516,  0.55251938])}]

In [19]:
def average_cluster(c):
    
    
    C = np.sum(c['cg'])
    C = C/abs(c['cg'])
 
    return C
        

In [20]:
average_cluster(c[0])

array([ 2.32329257,  2.93463235,  1.80547462, 32.02711354,  1.76361245])

In [ ]:
def average_genes():
    